In [ ]:
import numpy as np
import pandas as pd
import nd2reader
import matplotlib.pyplot as plt
import holoviews as hv
from holoviews.operation.datashader import regrid
import skimage.filters
import skimage.feature
import scipy.ndimage
import peakutils
from tqdm import tnrange, tqdm_notebook
import dask
import dask.array as da
import distributed
from distributed import Client, LocalCluster, progress
from dask_jobqueue import SLURMCluster
from cytoolz import partial, compose
from itertools import repeat
from glob import glob
import cachetools
import numpy_indexed
import pickle
import warnings
import os
from numbers import Integral
from dask.delayed import Delayed

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# from segmentation import *
# from util import *
# from matriarch_stub import *
import segmentation
import matriarch_stub

In [ ]:
plt.rcParams["figure.figsize"] = (8, 8)
hv.extension("bokeh")

# Run

In [ ]:
dask.config.config["distributed"]["scheduler"]["allowed-failures"] = 20
# dask.config.config['distributed']['worker']['memory'] = {'target': 0.4,
#                                                         'spill': 0.5,
#                                                         'pause': 0.9,
#                                                         'terminate': 0.95}

In [ ]:
cluster = SLURMCluster(
    queue="short",
    walltime="03:00:00",
    memory="8GB",
    local_directory="/tmp",
    log_directory="/home/jqs1/projects/molecule-counting/log",
    cores=1,
    processes=1,
)
# diagnostics_port=('127.0.0.1', 8787),
# env_extra=['export PYTHONPATH=\"/home/jqs1/projects/matriarch\"'])
client = Client(cluster)

In [ ]:
cluster

In [ ]:
cluster.scale(0)

## Run

In [ ]:
funcs = {"mean": np.mean}  # ,
#'median': np.median}
#'p0.05': partial(np.percentile, q=5),
#'p0.20': partial(np.percentile, q=20),
#'p0.70': partial(np.percentile, q=70),
#'p0.95': partial(np.percentile, q=95)}

In [ ]:
base_filename = "/n/scratch2/jqs1/fidelity/190313"
fluorescence_filenames = (
    glob(os.path.join(base_filename, "fluorescence/*.nd2"))
    + glob("/n/scratch2/jqs1/fidelity/190325/fluorescence/*/*.nd2")
    + glob("/n/scratch2/jqs1/fidelity/190326/*.nd2")
)
phase_filenames = glob(os.path.join(base_filename, "phase/*_0001.nd2")) + glob(
    "/n/scratch2/jqs1/fidelity/190325/phase/*/*_0001.nd2"
)
sandwich_filenames = glob(os.path.join(base_filename, "sandwich/*_0001.nd2"))

In [ ]:
dark_frames = segmentation.nd2_to_dask(
    os.path.join(base_filename, "calibration/dark_100ms.nd2"), 0, 0
)
dark_frame = dark_frames.mean(axis=0)
# TODO: hack
# dark_frame = dark_frame.compute()
# dark_frame = client.persist(dark_frame)
# dark_frame = client.scatter(dark_frame, broadcast=True)
dark_frame = dark_frame.to_delayed()[0, 0]

In [ ]:
flat_fields = {}
for filename in glob(os.path.join(base_filename, "calibration/*flat*100ms*.nd2")):
    channel = segmentation.get_nd2_reader(filename).metadata["channels"][0]
    flat_field = segmentation.nd2_to_dask(filename, 0, 0).mean(axis=0)
    # TODO: hack
    # flat_field = flat_field.compute()
    # flat_field = client.scatter(flat_field, broadcast=True)
    # flat_field = client.persist(flat_field)
    flat_field = flat_field.to_delayed()[0, 0]
    flat_fields[channel] = flat_field

In [ ]:
dark_frame = None
flat_fields = {}

In [ ]:
data_graph = {}
for photobleaching_filename in fluorescence_filenames[:]:
    data_graph[photobleaching_filename] = segmentation.process_file(
        funcs, photobleaching_filename, dark_frame=dark_frame, flat_fields=flat_fields
    )

for photobleaching_filename in phase_filenames[:]:
    segmentation_filename = photobleaching_filename.replace("_0001.nd2", ".nd2")
    data_graph[segmentation_filename] = segmentation.process_file(
        funcs,
        photobleaching_filename,
        segmentation_filename=segmentation_filename,
        dark_frame=dark_frame,
        flat_fields=flat_fields,
    )

for initial_filename in sandwich_filenames[:]:
    segmentation_filename = initial_filename.replace("_0001.nd2", ".nd2")
    photobleaching_filename = initial_filename.replace("_0001.nd2", "_0002.nd2")
    final_filename = initial_filename.replace("_0001.nd2", "_0003.nd2")
    data_graph[segmentation_filename] = segmentation.process_file(
        funcs,
        photobleaching_filename,
        segmentation_filename=segmentation_filename,
        initial_filename=initial_filename,
        final_filename=final_filename,
        dark_frame=dark_frame,
        flat_fields=flat_fields,
    )

In [ ]:
# split up computes so we can gather results from multiple workers
# (otherwise the single worker assembling the dict will run out of memory)
data_futures = {k: client.compute(v) for k, v in data_graph.items()}
data_futures["_calibration"] = client.compute(
    {"dark_frame": dark_frame, "flat_fields": flat_fields}
)

## Save data

In [ ]:
data = client.gather(data_futures)

In [ ]:
data.keys()

In [ ]:
with open("/home/jqs1/scratch/190328photobleaching_flatcorr.pickle", "wb") as f:
    pickle.dump(data, f)

In [ ]:
# time: 96593s
# 65% map_over_labels, 30% get_nd2_frame
# time: 109885s (3/22)
# 56% map_over_labels 43% get_nd2_frame
# time: 83334s (3/26, only fluor)
# 88% map_over_labels 9% get_nd2_frame

In [ ]:
%%output size=200
hv.HoloMap(
    {
        filename: RevImage(permute_labels(data[filename][0]["labels"]))
        for filename in data.keys()
    }
)

# Segmentation practice

In [ ]:
filenames = [
    "/n/scratch2/jqs1/fidelity/190326/190326_mkate_nochlor_wateragarpad_100ms_50pct_laser.nd2",
    "/n/scratch2/jqs1/fidelity/190326/190326_GFP_chlor_wateragarpad_100ms_100pct_laser.nd2",
    "/n/scratch2/jqs1/fidelity/190326/190326_mkate_chlor_wateragarpad_100ms_100pct_laser.nd2",
    "/n/scratch2/jqs1/fidelity/190325/fluorescence/mkate_LBagarpad_100ms_100pct_laser/190325_mkate_lbagarpad_100ms_100pct_laser.nd2",
]

In [ ]:
frame = segmentation.get_nd2_frame(
    "/n/scratch2/jqs1/fidelity/190313/fluorescence/190313_mkate_100ms_50pct_laser.nd2",
    8,
    0,
    0,
)

In [ ]:
plt.imshow(frame)

In [ ]:
%%output size=150
regrid(diag["img"]) + diag["img_thresh_masked"]

In [ ]:
diag = {}
labels = segmentation.segment(frame[500:1500, 500:1500], diagnostics=diag)

In [ ]:
plt.imshow(segmentation.permute_labels(labels))

In [ ]:
hv.operation.contours?

In [ ]:
hv.operation.contours(segmentation.RevImage(labels), levels=30)

In [ ]:
hv.operation.contours(segmentation.RevImage(labels), levels=np.arange(labels.max() - 1))

In [ ]:
%%output size=100
(
    regrid(diag["img"])
    + regrid(diag["img_k1"])
    + regrid(diag["img_k1_frangi"])
    + regrid(diag["img_k1_frangi_thresh"])
    + regrid(diag["clean_seeds"])
    + regrid(diag["watershed_labels_permuted"])
).cols(3)

In [ ]:
skimage.filters.threshold_local??

In [ ]:
img_k1_frangi = diag["img_k1_frangi"].data[800:1000, 500:700]
mask = diag["mask"].data[800:1000, 500:700]
img_k1_frangi_uint = skimage.img_as_uint(img_k1_frangi)
# selem = skimage.morphology.disk(20)
# img_k1_frangi_thresh = skimage.filters.rank.otsu(img_k1_frangi_uint, selem)
# img_k1_frangi_thresh_blurred = skimage.filters.gaussian(img_k1_frangi_thresh, 0)
# img_k1_frangi_thresh = skimage.filters.threshold_local(img_k1_frangi, block_size=21, mode='nearest')
# img_k1_frangi_thresh = skimage.filters.threshold_local(img_k1_frangi, block_size=101, mode='nearest', method='generic', param=skimage.filters.threshold_yen)
# img_k1_frangi_thresh = skimage.filters.threshold_local(img_k1_frangi, block_size=21, mode='nearest', method='median')
img_k1_frangi_thresh = skimage.filters.threshold_local(
    img_k1_frangi, block_size=23, mode="nearest"
)
# img_k1_frangi_thresh = np.exp(skimage.filters.threshold_local(np.log(img_k1_frangi), block_size=23, mode='nearest', method='mean'))
# img_thresh = img_k1_frangi > img_k1_frangi_thresh_blurred
img_thresh = img_k1_frangi > img_k1_frangi_thresh
img_thresh_masked = img_thresh * mask
# img_thresh_eroded = (skimage.morphology.erosion)(img_thresh_masked)
plt.figure(figsize=(20, 20))
plt.imshow(img_thresh_masked)

In [ ]:
selem = skimage.morphology.disk(10)
frangi_int = skimage.img_as_uint(frangi)
local_otsu = skimage.filters.rank.otsu(frangi_int, selem)
# plt.imshow(frangi_int - local_otsu)
plt.imshow(local_otsu)

In [ ]:
skimage.filters.threshold_otsu(diag["img_k2_frangi"].data)

In [ ]:
frangi = diag["img_k2_frangi"].data[:600, :600]

In [ ]:
plt.imshow(np.log(frangi))

In [ ]:
selem = skimage.morphology.disk(10)
frangi_int = skimage.img_as_uint(frangi)
local_otsu = skimage.filters.rank.otsu(frangi_int, selem)
# plt.imshow(frangi_int - local_otsu)
plt.imshow(local_otsu)

In [ ]:
plt.imshow(skimage.filters.gaussian(local_otsu, 10))

In [ ]:
plt.hist(frangi_int.flat, bins=100, log=True)

In [ ]:
plt.hist(local_otsu.flat, bins=100, log=True)

In [ ]:
plt.figure(figsize=(20, 20))
plt.imshow(local_otsu > 2000)

In [ ]:
plt.figure(figsize=(20, 20))
plt.imshow(frangi_int > 500)

In [ ]:
plt.figure(figsize=(20, 20))
plt.imshow(frangi_int > skimage.filters.gaussian(local_otsu, 10) * 0)

In [ ]:
frangi.max()

In [ ]:
plt.scatter(frangi.flat, local_otsu.flat)

In [ ]:
plt.imshow(
    diag["img_k2_frangi"].data
    > skimage.filters.threshold_otsu(diag["img_k2_frangi"].data)
)

In [ ]:
plt.hist(diag["img_k2_frangi_thresh"].data.flat, bins=100, log=True)

In [ ]:
diag.keys()

In [ ]:
diag["img_k2"].data

In [ ]:
frangi = diag["img_k2_frangi"].data

In [ ]:
selem = skimage.morphology.disk(10)

local_otsu = skimage.filters.rank.otsu(frangi, selem)

In [ ]:
plt.imshow(frangi > local_otsu)

In [ ]:
plt.imshow(local_otsu)